In [2]:
!pip install pandas openai

In [3]:
import pandas as pd
import json
from openai import OpenAI


In [4]:
from google.colab import userdata
from openai import OpenAI

client = OpenAI(
    api_key=userdata.get("OPENAI_API_KEY")
)


In [5]:
!pip install gspread oauth2client


In [6]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials


In [7]:
import gspread
from google.auth import default

SCOPES = [
    "https://www.googleapis.com/auth/spreadsheets",
    "https://www.googleapis.com/auth/drive"
]

creds, _ = default(scopes=SCOPES)
gc = gspread.authorize(creds)


In [8]:
!pip install gspread google-auth


In [10]:
from google.colab import auth
auth.authenticate_user()


In [13]:




creds, _ = default()
gc = gspread.authorize(creds)

SHEET_URL = "https://docs.google.com/spreadsheets/d/1wUr81U6hk7WYqxkmdQlHIAZ7JhBRjaGPDQJMObOjZZo/"
sh = gc.open_by_url(SHEET_URL)

destination_master = sh.worksheet("Destination_Master").get_all_records()
packages_master = sh.worksheet("Packages_Master").get_all_records()
daywise_rows = sh.worksheet("Daywise_Itinerary").get_all_records()
inc_exc_rows = sh.worksheet("Inclusions_Exclusions").get_all_records()

print("Loaded:")
print("Destinations:", len(destination_master))
print("Packages:", len(packages_master))
print("Daywise:", len(daywise_rows))
print("Inc/Exc:", len(inc_exc_rows))



Loaded:
Destinations: 1
Packages: 1
Daywise: 7
Inc/Exc: 8


In [14]:
# Destination index
dest_index = {
    d["Destination_Code"].strip(): d
    for d in destination_master
}

# Daywise itinerary index
itinerary_index = defaultdict(list)

for row in daywise_rows:
    itinerary_index[row["Destination_ID"].strip()].append({
        "day": int(row["Day"]),
        "description": row["Description"].strip()
    })

# Inclusions / exclusions index
inc_exc_index = defaultdict(lambda: {"inclusions": [], "exclusions": []})

for row in inc_exc_rows:
    pid = row["Destination_ID"].strip()
    if row.get("Inclusions"):
        inc_exc_index[pid]["inclusions"].append(row["Inclusions"].strip())
    if row.get("Exclusions"):
        inc_exc_index[pid]["exclusions"].append(row["Exclusions"].strip())

print("Indexes ready")


Indexes ready


In [15]:
def slugify(text):
    return text.lower().replace(" ", "-").replace("/", "").strip()

def derive_budget(price):
    try:
        p = int(price)
        if p < 60000:
            return "Budget"
        elif p < 120000:
            return "Mid"
        else:
            return "Premium"
    except:
        return "Mid"


In [16]:
final_packages = []

for pkg in packages_master:
    pkg_id = pkg["Destination_ID"].strip()
    dest_code = pkg_id[:3]

    destination = dest_index.get(dest_code)

    if not destination:
        print("⚠️ Missing destination for", pkg_id)
        continue

    itinerary = sorted(
        itinerary_index.get(pkg_id, []),
        key=lambda x: x["day"]
    )

    nights = len(itinerary)
    days = nights + 1 if nights else None

    price_str = str(pkg.get("Price_Range_INR", "")).strip()

    final_packages.append({
        "Destination_ID": pkg_id,
        "Destination_Name": destination["Destination_Name"],
        "Overview": f"Experience the best of {destination['Destination_Name']} with Travelzada.",
        "Duration": f"{nights} Nights / {days} Days" if nights else None,
        "Budget_Category": derive_budget(price_str),
        "Price_Range_INR": price_str,
        "Group_Size": 2,
        "Inclusions": inc_exc_index[pkg_id]["inclusions"],
        "Exclusions": inc_exc_index[pkg_id]["exclusions"],
        "Day_Wise_Itinerary": itinerary,
        "Slug": slugify(f"{destination['Destination_Name']}-{pkg_id}"),
        "Booking_URL": f"https://travelzada.com/packages/{pkg_id}",
        "Duration_Nights": nights,
        "Duration_Days": days
    })

print("Final packages:", len(final_packages))


Final packages: 1


In [17]:
with open("travelzada_packages.json", "w") as f:
    json.dump(final_packages, f, indent=2)

print("✅ JSON generated successfully")


✅ JSON generated successfully
